# Processing coreference using a set of PERIODICAL's biographies

Requirements

Citation
    spacy library
    coref library: "fastcoref"
    Repository: https://github.com/shon-otmazgin/fastcoref#demo
    
    @inproceedings{Otmazgin2022FcorefFA,
      title={F-coref: Fast, Accurate and Easy to Use Coreference Resolution},
      author={Shon Otmazgin and Arie Cattan and Yoav Goldberg},
      booktitle={AACL},
      year={2022}
    }

In [1]:
from fastcoref import spacy_component
import spacy

nlp = spacy.load("en_core_web_trf")
# smaller dataset 
# nlp = spacy.load("en_core_web_sm")

# for entity recognition
# entity linking
import spacyfishing
# specify configuration:
nlp.add_pipe("entityfishing", config={"extra_info": True})

# add to the pipeline fastcoref
import spacy_transformers
from spacy.tokens import Doc
nlp.add_pipe("fastcoref")

print(nlp.pipe_names)

/data/user-data/amt557/meetups_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
01/12/2024 16:52:46 - INFO - 	 missing_keys: []
01/12/2024 16:52:46 - INFO - 	 unexpected_keys: []
01/12/2024 16:52:46 - INFO - 	 mismatched_keys: []
01/12/2024 16:52:46 - INFO - 	 error_msgs: []
01/12/2024 16:52:46 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'entityfishing', 'fastcoref']


In [2]:
import os
import pandas as pd
import re
import numpy as np
import glob
# to eliminate accents
# from unidecode import unidecode
# to clear memory
import gc
pd.options.mode.copy_on_write = True
# time execution
from timeit import default_timer as timer

In [3]:
# File with list of biographies to process
# headers = ['qid', 'dbpedia_id', 'resource']
# Read the CSV file into a DataFrame
df = pd.read_csv('matching_bios.csv')
df['dbpedia_id'] = df['dbpedia_id'].astype(str)
df = df.drop_duplicates(subset=['dbpedia_id'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   qid         40 non-null     object
 1   dbpedia_id  40 non-null     object
 2   resource    40 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


,qid,dbpedia_id,resource
0,Q254,33163,http://dbpedia.org/resource/Wolfgang_Amadeus_M...
1,Q130759,99636,http://dbpedia.org/resource/Christoph_Willibal...
2,Q364020,376771,http://dbpedia.org/resource/Étienne_Méhul
3,Q168485,261539,http://dbpedia.org/resource/Gaspare_Spontini
4,Q154203,354604,http://dbpedia.org/resource/Albert_Lortzing


In [4]:
# List files available
def find_files(folder_path, file_pattern):
    """
    Find files in a folder based on a specified pattern.

    Parameters:
    - folder_path: The path to the folder.
    - file_pattern: The pattern to match for file names (e.g., '*.txt').

    Returns:
    - A list of file paths that match the specified pattern.
    """
    search_pattern = f"{folder_path}/{file_pattern}"
    files = glob.glob(search_pattern)
    # next line list only the file names, if commented then the function
    # retunrs the entire path
    # file_names = [os.path.basename(file) for file in files]
    return files

# Example usage:
# folder_path = '../meetups_pilot/cacheCoref-fastcoref'
# file_pattern = '31281773_*.coref'

# found_files = find_files(folder_path, file_pattern)

# if found_files:
#     print("Found files:")
#     for file_path in found_files:
#         print(file_path)
# else:
#     print("No files found.")

In [12]:
match = re.search(r'_(\d+)\.coref', '../meetups_pilot/cacheCoref-fastcoref/31281773_9.coref')
print(int(match.group(1)) if match else None)

paragraph_number = int(re.search(r'_(\d+)\.coref', file_path).group(1)) if re.search(r'_(\d+)\.coref', file_path) else None
print(paragraph_number)

9
9


In [5]:
# For this development we use the coference objects created while executing the MEETUPS pipeline
# it is also possible to process the text and obtain the object if other/new text has to be process
folder_name = '../meetups_pilot/cacheCoref-fastcoref'
# list of coreferenced texts
no_found = []
for bio in df.itertuples():
    coref_text = []
    file_name_pattern = str(bio.dbpedia_id)+'_*.coref'
    found_files = find_files(folder_name, file_name_pattern)
    if found_files:
        print("Found files... "+str(bio.dbpedia_id))
        for file_path in found_files:
            print(file_path)

            try:
                # spacy object, including the coreference step in the pipeline
                doc = Doc(nlp.vocab).from_disk(file_path)
                # paragraph number, extracted from the file_path 
                paragraph_number = int(re.search(r'_(\d+)\.coref', file_path).group(1)) if re.search(r'_(\d+)\.coref', file_path) else None
                coref_text.append([doc._.resolved_text,paragraph_number])
            except Exception as e: 
                print("ERROR: ", bio)
                print(e)
                continue
    else:
        print("No files found.")
        no_found.append(str(bio.dbpedia_id))

    # save results
    if len(coref_text) > 0:
        # Create a DataFrame from the list of lists
        df_result = pd.DataFrame(coref_text, columns=['coref_text', 'paragraph_number'])
        # Save the DataFrame to a CSV file
        df_result.to_csv('results-coref-text/'+str(bio.dbpedia_id)+'.csv', index=False)

Found files... 33163
../meetups_pilot/cacheCoref-fastcoref/33163_0.coref
../meetups_pilot/cacheCoref-fastcoref/33163_1.coref
../meetups_pilot/cacheCoref-fastcoref/33163_4.coref
../meetups_pilot/cacheCoref-fastcoref/33163_5.coref
../meetups_pilot/cacheCoref-fastcoref/33163_6.coref
../meetups_pilot/cacheCoref-fastcoref/33163_7.coref
../meetups_pilot/cacheCoref-fastcoref/33163_8.coref
../meetups_pilot/cacheCoref-fastcoref/33163_9.coref
../meetups_pilot/cacheCoref-fastcoref/33163_10.coref
../meetups_pilot/cacheCoref-fastcoref/33163_11.coref
../meetups_pilot/cacheCoref-fastcoref/33163_12.coref
../meetups_pilot/cacheCoref-fastcoref/33163_13.coref
../meetups_pilot/cacheCoref-fastcoref/33163_14.coref
../meetups_pilot/cacheCoref-fastcoref/33163_15.coref
../meetups_pilot/cacheCoref-fastcoref/33163_16.coref
../meetups_pilot/cacheCoref-fastcoref/33163_18.coref
../meetups_pilot/cacheCoref-fastcoref/33163_19.coref
../meetups_pilot/cacheCoref-fastcoref/33163_20.coref
../meetups_pilot/cacheCoref-fastc

In [6]:
print(len(no_found))
print(no_found)
# Create a DataFrame from the list of lists
df_no_found = pd.DataFrame(no_found, columns=['dbpedia_id'])
# Save the DataFrame to a CSV file
df_no_found.to_csv('no-found-list.csv', index=False)

5
['2444917', '10085', '21511', '2843958', '45181']


In [7]:
# add original text
for bio in df.itertuples():
    try:
        # read indexed paragraphs
        df = pd.read_csv('../meetups_pilot/indexedParagraphs/'+str(bio.dbpedia_id)+'.csv')
        df.drop(columns=['section'], inplace=True)
        df.rename(columns={'paragraph': 'ori_text'}, inplace=True)
        df_coref = pd.read_csv('results-coref-text/'+str(bio.dbpedia_id)+'.csv')
        df_coref.rename(columns={'paragraph_number': 'paragraphIndex'}, inplace=True)
        
        merged_df = pd.merge(df, df_coref, on='paragraphIndex')
        merged_df.to_csv('results-coref-text/'+str(bio.dbpedia_id)+'.csv', index=False)
    except Exception as e: 
        print("ERROR: ", bio)
        print(e)
        continue

In [20]:
# read each paragraph in the biography
for bio in no_found:
    try:
        coref_text = []
        df_prg = pd.read_csv('../meetups_pilot/indexedParagraphs/'+str(bio)+'.csv')
        for prg in df_prg.itertuples():
            doc = nlp(prg.paragraph, component_cfg={"fastcoref":{"resolve_text":True}})
            coref_text.append([prg.paragraph,prg.paragraphIndex,doc._.resolved_text])
            print('doc to disk: ', bio+'_'+str(prg.paragraphIndex)+'.coref')
            doc.to_disk("doc-objects/"+bio+'_'+str(prg.paragraphIndex)+'.coref')
    
         # save results
        if len(coref_text) > 0:
            # Create a DataFrame from the list of lists
            df_result = pd.DataFrame(coref_text, columns=['ori_text', 'paragraphIndex','coref_text'])
            # Save the DataFrame to a CSV file
            df_result.to_csv('results-coref-text/'+str(bio)+'.csv', index=False)
    except Exception as e: 
        print("ERROR: ", bio)
        print(e)
        continue

ERROR:  183747
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/183747.csv'
ERROR:  45190
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/45190.csv'
ERROR:  328911
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/328911.csv'
ERROR:  10823
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/10823.csv'
ERROR:  14135
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/14135.csv'
ERROR:  12775
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/12775.csv'
ERROR:  44981
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/44981.csv'
ERROR:  27808
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/27808.csv'
ERROR:  631987
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/631987.csv'
ERROR:  339480
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/339480.csv'
ERROR:  2140

01/12/2024 15:34:47 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:00<00:00, 121.30 examples/s]
01/12/2024 15:34:51 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00, 10.38it/s]


doc to disk:  44887_0.coref


01/12/2024 15:34:52 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:00<00:00, 71.32 examples/s]
01/12/2024 15:34:56 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00, 14.15it/s]


doc to disk:  44887_1.coref


...

doc to disk:  45181_85.coref


Map: 100%|██████████| 1/1 [00:00<00:00, 155.44 examples/s]
01/12/2024 16:04:55 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00, 38.87it/s]


doc to disk:  45181_86.coref


01/12/2024 16:04:56 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:00<00:00, 161.49 examples/s]
01/12/2024 16:04:59 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00, 30.46it/s]

doc to disk:  45181_87.coref
ERROR:  2363
[Errno 2] No such file or directory: '../meetups_pilot/indexedParagraphs/2363.csv'
